根據把首頁往下滑，發現網站會對server送一個GET請求下一個30篇文章
- 2018(G) https://ithelp.ithome.com.tw/articles/10203216
- 2019(G) https://tlyu0419.github.io/2019/04/06/Crawl-Dcard/
- 2020 https://blog.jiatool.com/posts/dcard_api_v2/

### API root

- 看板資訊 http://dcard.tw/_api/forums
- 文章資訊 http://dcard.tw/_api/forums/{看板名稱}/posts
- 文章內文	http://dcard.tw/_api/posts/{文章編號}
- 文章內引用連結	http://dcard.tw/_api/posts/{文章編號}/links
- 文章內留言	http://dcard.tw/_api/posts/{文章編號}/comments

### API wirte
- 文章數量 https://www.dcard.tw/_api/posts?limit=100
- 文章分頁 http://dcard.tw/_api/posts/?before=文章id (before 回傳由新到舊 after舊到新)
- 文章留言 http://dcard.tw/_api/posts/文章id/comments
- 留言分頁 http://dcard.tw/_api/posts/文章id/comments?after=floor數字
- 熱門文章 https://dcard.tw/_api/posts?popular=true

### others
文章資訊(meta) 與 文章內留言 預設使用熱門度 (popularity) 作為排序，而且一次請求 (request) 中只回應 30 筆。
* 想要讓這兩項使用 時間 排序，可在 GET 參數中加入 popular=false
* 欲取得更多的 文章資訊(meta)，可以使用 before={某文章編號}來獲得早於 #{某文章編號} 的另外 30 筆 資訊。
* 欲取得更多的 文章內留言，可以使用 after={某樓層} 來獲得大於 #{某樓層} 的另外 30 筆 留言。

In [1]:
import requests
import json
import pandas as pd    # 引用套件並縮寫為 pd
from bs4 import BeautifulSoup
# https://www.dcard.tw/f/makeup

GET 美妝版首頁，Use BeautifulSoup with html, wrtie a txt

 

In [2]:
p = requests.Session()                             # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/f/makeup")  # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
#sel

#for link in sel:
#    a.append(link["href"])
    #print('https://www.dcard.tw'+link.get('href'))

In [3]:
# slope the title
def Catch(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [4]:
# KeyError: 'id'
# Oh！文章不見了 這篇文章已被作者刪除。 按這裡 看更多文章吧
# https://www.dcard.tw/f/makeup/p/235079399
# Catch(235079399)

In [5]:
# # 1w : 234227720
# # 2w : 233254532
# url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=233254532" 

In [6]:
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=1"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True) 
#df
print(str(int(df['id'].tail(1))) )

235114349


In [7]:
count = 0
delay_choices = [1, 3, 5]  # delay second
delay = random.choice(delay_choices) 

for j in range(1):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    print(last)
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        count = count+1
        print('data reading...',count,'id:',rejs[i]['id'],rejs[i]['title'])
    delay = random.choice(delay_choices)
    time.sleep(delay)  
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

235114349
data reading... 1 id: 235114259 護唇膏 求推薦！！（有選手）
data reading... 2 id: 235114114 #請益 Ga水幕斯遮瑕精萃選色～
data reading... 3 id: 235113758 #請益 關於蘭蔻小黑瓶的下個步驟
data reading... 4 id: 235113696 Pat McGrath Mothership VII Divine Rose II 玫瑰二代盤試色
data reading... 5 id: 235113657 #請益 想請問寶雅員工這個到底多少錢？
data reading... 6 id: 235113567 #求評價！新北霧眉
data reading... 7 id: 235113469 啊哈
data reading... 8 id: 235113426 #請益 專櫃護手霜推薦
data reading... 9 id: 235113135 化妝心態
data reading... 10 id: 235112281 新發現 粉底液開蓋 用到最後剩一點點
data reading... 11 id: 235112270 #分享 高雄霧眉分享 #三個月後
data reading... 12 id: 235112267 #分享 我的化妝包分享 近期購入的新品 專櫃開架
data reading... 13 id: 235111960 #分享 2021年一月🈳️🈳️賞🎊
data reading... 14 id: 235111914 #討論 屈臣氏的卓沿白金
data reading... 15 id: 235111756 #問 fitme遮瑕色號
data reading... 16 id: 235111687 #分享 花錢買快樂
data reading... 17 id: 235111631 #請益 請問未開封的唇釉可以換貨嗎？
data reading... 18 id: 235111592 #分享 在屈臣氏撿到便宜！
data reading... 19 id: 235111543 #情報 紅水黑大方🌟🌟LM限定款氣墊
data reading... 20 id: 235111534 #試色 GA絲絨唇萃 威尼斯日落暖棕新色分享


,id,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,235114349,台中縫雙眼皮,一直以來都有想要美美的雙眼皮\n最今有很大的衝動想去小整\n爬了多文\n現在看台中*聖(楊醫...,一直以來都有想要美美的雙眼皮，最今有很大的衝動想去小整，爬了多文，現在看台中*聖（楊醫師）評...,2021-01-05T02:45:17.197Z,2021-01-05T02:45:17.197Z,美妝,makeup,F,NaN
1,235114259,護唇膏 求推薦！！（有選手）,男友吃A酸好幾個月 副作用是嘴唇會乾到裂\n他本來有DHC的橄欖油護唇膏\n但好像不喜歡味道...,男友吃A酸好幾個月 副作用是嘴唇會乾到裂，他本來有DHC的橄欖油護唇膏，但好像不喜歡味道而且...,2021-01-05T02:21:20.917Z,2021-01-05T02:21:43.863Z,美妝,makeup,F,NaN
2,235114114,#請益 Ga水幕斯遮瑕精萃選色～,最近在觀望Ga的遮瑕精萃，想問水幕斯粉底液和氣墊粉餅都用3號，如果要遮黑眼圈和痘疤要選幾號遮...,最近在觀望Ga的遮瑕精萃，想問水幕斯粉底液和氣墊粉餅都用3號，如果要遮黑眼圈和痘疤要選幾號遮...,2021-01-05T01:41:00.866Z,2021-01-05T01:41:00.866Z,美妝,makeup,F,NaN
3,235113758,#請益 關於蘭蔻小黑瓶的下個步驟,請問大家蘭蔻小黑瓶擦完都接著擦什麼保養品呢\n求推薦🥺 謝謝🙏🏻,請問大家蘭蔻小黑瓶擦完都接著擦什麼保養品呢，求推薦🥺 謝謝,2021-01-04T20:46:08.725Z,2021-01-04T20:46:08.725Z,美妝,makeup,F,NaN
4,235113696,Pat McGrath Mothership VII Divine Rose II 玫瑰二代盤試色,https://i.imgur.com/Jbw3INz.jpg\n今天要分享的是patmcg...,今天要分享的是patmcgrath的十色眼影盤第八代之前natasha denona的液態眼...,2021-01-04T19:57:12.283Z,2021-01-04T19:57:12.283Z,美妝,makeup,M,NaN
...,...,...,...,...,...,...,...,...,...,...
96,235102363,#試色 Make up for ever 藝術家-星球,這兩隻火星和土星是贈品啦 我個人不會用這個顏色\n塗起來真的閃到爆炸 從裡閃到外 \n適合P...,這兩隻火星和土星是贈品啦 我個人不會用這個顏色，塗起來真的閃到爆炸 從裡閃到外，適合Part...,2021-01-03T06:39:03.035Z,2021-01-03T10:13:05.842Z,美妝,makeup,F,NaN
97,235102318,#分享 很給力💪敏感肌的冬天保養戰將們,2021首發就現給美妝版了～💕\n今天來分享微敏感肌人用了一陣子的好物\n跨年這幾天冷到爆我...,2021首發就現給美妝版了～今天來分享微敏感肌人用了一陣子的好物，跨年這幾天冷到爆我的膚況也...,2021-01-03T06:27:40.869Z,2021-01-03T06:27:40.869Z,美妝,makeup,F,NaN
98,235102316,有人買了這個系列嗎,整個被這個外殼燒到👀\n但沒買過他家的唇膏，想先問一下大家對他們的評價~\n我目前買過的是\...,整個被這個外殼燒到，但沒買過他家的唇膏，想先問一下大家對他們的評價~，我目前買過的是，迪奧9...,2021-01-03T06:27:28.013Z,2021-01-03T11:53:29.180Z,美妝,makeup,F,NaN
99,235102287,#請益 想問大家對純素保養的看法！,大家好因為課堂ㄉ關係想要問問大家有沒有聽過純素保養！那在台灣其實很多品牌都開始做純素保養想問...,大家好因為課堂ㄉ關係想要問問大家有沒有聽過純素保養！那在台灣其實很多品牌都開始做純素保養想問...,2021-01-03T06:20:08.628Z,2021-01-03T06:20:08.628Z,美妝,makeup,F,NaN


# ---------------正式版本分隔線---------------
- 測試版本代號：1.2  2020
- 正式版本代號：2.03  2021
- 開發人員： EGG Jerry Three


dva.x_x 版本開發

In [66]:
#找到 dav.x_x 發表過的文章

p = requests.Session()                             # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/@dva.x_x")  # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
sel
a=[]
for link in sel:
    a.append(link["href"])
    print(link.get('href')[-9:])
    print('https://www.dcard.tw'+link.get('href'))

235103419
https://www.dcard.tw/f/makeup/p/235103419
234999065
https://www.dcard.tw/f/girl/p/234999065
234941909
https://www.dcard.tw/f/vehicle/p/234941909
234918075
https://www.dcard.tw/f/boutique/p/234918075
234912821
https://www.dcard.tw/f/makeup/p/234912821
234773210
https://www.dcard.tw/f/boutique/p/234773210
234748104
https://www.dcard.tw/f/3c/p/234748104
234720985
https://www.dcard.tw/f/boutique/p/234720985
234700673
https://www.dcard.tw/f/makeup/p/234700673
234688353
https://www.dcard.tw/f/makeup/p/234688353
234659616
https://www.dcard.tw/f/makeup/p/234659616
234615352
https://www.dcard.tw/f/makeup/p/234615352
234603725
https://www.dcard.tw/f/makeup/p/234603725
234516036
https://www.dcard.tw/f/makeup/p/234516036
234329048
https://www.dcard.tw/f/makeup/p/234329048
234278177
https://www.dcard.tw/f/boutique/p/234278177
234231596
https://www.dcard.tw/f/makeup/p/234231596
234225704
https://www.dcard.tw/f/makeup/p/234225704
234161273
https://www.dcard.tw/f/boutique/p/234161273
2340467

In [52]:
# slope the title
def CatchUser(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'department':rejs['department'],  #for author
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'department', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'department':"",    #for author
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'department', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [55]:
# 熱門美妝版測試 (100筆)
url = 'https://www.dcard.tw/@dva.x_x'
#url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=5'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(CatchUser(rejs[i]['id']),ignore_index=True)   #0 - 29
print(df.shape)
#df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色
# 235066991

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [46]:
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=1"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(CatchUser(rejs[0]['id']),ignore_index=True) 
#df
print(str(int(df['id'].tail(1))) )

count = 0
delay_choices = [1, 3, 5]  # delay second
delay = random.choice(delay_choices) 

for j in range(1):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    print(last)
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        count = count+1
        print('data reading...',count,'id:',rejs[i]['id'],rejs[i]['title'])
    delay = random.choice(delay_choices)
    time.sleep(delay)  
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

235111465


In [ ]:
# slope the title
def CatchPost(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [34]:
#CatchPost(234999065)
#url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
url = 'https://www.dcard.tw/_api/posts/235103419'
resq = requests.get(url)
rejs = resq.json()
rejs

{'id': 235103419,
 'title': '試色6隻YSL小粉條跟玫瑰金蓋',
 'content': 'YSL玫瑰金系列唇彩試色來了\nhttps://i.imgur.com/qGBQoMx.jpg\n一口氣買了好幾隻\n因為到現場試色時發現每一隻都好美無法做決定\n只好把比較喜歡的顏色都收了\n剛好玫瑰金蓋跟小粉條的顏色都買一樣的\n給大家對比一下兩種質地同個色號的差別是什麼\n也順便給大家看看我滿額拿到的超美收納盒\n這6隻除了小粉條211之外\n其他5隻都是我自掏腰包購入\n絕對不是業配\n\n先附上電子發票跟櫃姐對話證明一下\nhttps://i.imgur.com/VHMM6YH.jpg\n\nhttps://i.imgur.com/qatX047.jpg\n偷偷推一下新竹巨城YSL的天使櫃姐 佩珊\n很親切也不用怕被冰_(:з」∠)_\n\n以下為超真實試色\n我盡量拍到跟肉眼看到是差不多的顏色\n希望能給你們一個參考依據\n-\n先來說說這次讓我很驚艷的 #玫瑰小粉條\nhttps://i.imgur.com/85u3VM2.jpg\n我覺得它的質地介於小金條跟小銀條之間\n小金條對我來說太乾\n小銀條對我來說又太輕透\n所以其實沒有很愛這兩款唇膏\n但這次的小粉條真的很對我的味\n滑順度優秀、顏色飽和度也適中\n唇色遮蓋力大約9成左右\nhttps://i.imgur.com/jmLvNWY.jpg\n小粉條乾唇人也能入手\n它一點都不乾而且也不會卡死皮\n顏色也不會陷進唇紋表現真的很優秀\n雖然不防沾染但是喝個水顏色都還會在\nhttps://i.imgur.com/9zqXy9W.jpg\n202跟211我覺得必買\n兩個顏色在日常使用都很百搭\n是個人人都能駕馭的紅棕色\n\n(滿分為五分)\n遮蓋力：4\n保濕度：3\n飽和度：4.5\n顯色度：4\n持久度：4\n香味：淡淡紅酒味\n價格：$1360\n-\n\n👉🏻再來是最多人敲碗的 #玫瑰金蓋\nhttps://i.imgur.com/acMFnnC.jpg\n這隻的飽和度比我想像中的好一點\n薄擦大概只能遮蓋7成左右的唇色\n厚擦至少能遮蓋8到9成\n以輕透的唇釉來說表現算不錯了\n它的成膜速度很快\n所以如果要畫染唇妝的話手腳要快一點\n不然後續要疊擦的話就會糊掉\n

In [20]:
url = 'https://www.dcard.tw/_api/posts/234912821'
resq = requests.get(url)
rejs = resq.json()
rejs

{'id': 234912821,
 'title': '幾隻愛馬仕唇膏隨意試色 順便偷渡愛馬仕潤唇膏',
 'content': '前陣子買了兩隻愛馬仕唇膏的熱賣色\n櫃哥還送了我試色卡\n其中一個顏色是33號經典色愛馬仕橘\n我有上嘴但看起來有點可怕就沒拍了\n(這顏色不是我這種凡夫俗子可以駕馭的( ´▽｀))\n大家可以參考我的手部試色XD\n\nhttps://i.imgur.com/YjevfFV.jpg\n\n手部試色\nhttps://i.imgur.com/gYf2D4D.jpg\n\n85號色就是很有氣勢的高冷葡萄酒紅色\n64號是比較活力日常的桃紅色\n這兩個顏色都顯白\n48號日常的煙燻玫瑰色\n沒有缺點 就是好看百搭絕不踩雷\n21號也很日常百搭 但因為是滋潤款會有一點不均勻\n\nhttps://i.imgur.com/kNN0fnC.jpg\nhttps://i.imgur.com/y1r1bLQ.jpg\nhttps://i.imgur.com/z3vRRFY.jpg\n上面三隻都是霧面的也是我覺得最值得收的質地\n雖然它是霧面的但是它超級絲滑\n它真的是我用過最絲滑的霧面唇彩\n而且超級輕薄沒有什麼存在感\n上嘴後它不是全霧面的還是有一點絲緞光\n不會乾也不會覺得厚重\n即使上妝一陣子也不會卡死皮或是顏色陷進唇紋\n喝水講話顏色都還會在\n吃完巧克力吐司顏色也還在XD\n整體來說就是表現很優秀的霧面唇彩\n\n\nhttps://i.imgur.com/oDoCgva.jpg\n這隻的質地是水潤光感的\n相對的顏色飽和度跟顯色度就沒這麼高\n唇色較深的人一定要先用粉底蓋一下唇色\n大概來回塗個兩次才會很顯色\n所以如果唇色深的人我建議買霧面款的\n或是滋潤款買顏色較重的\n\n用了一陣子下來最愛的還是48號\n真的太有氣質太溫柔了( ´ ▽ ` )ﾉ\n包裝一樣有愛馬仕的小橘盒跟防塵袋\n就跟買他們家包包一模一樣\n而且蓋子還是磁吸是的很有質感\n唇膏拿起來也是有份量的\n比之前在廣告宣傳圖上看到的有質感多了\n他們家的唇膏用完了可以只買替換芯就好\n\n信義新光三越A8有快閃櫃喔\n\nhttps://i.imgur.com/IwoVbBU.jpg\n容我偷渡一下潤唇膏(⁎⁍̴̛ᴗ⁍̴̛⁎)\n它的滋潤度中規中矩\n跟DHC的潤度差

In [ ]:
# url = 'https://www.dcard.tw/_api/posts/235103302'
resq = requests.get(url)
rejs = resq.json()
rejs

# --------------- dva.x_x 版本分隔線 ---------------
- 測試版本代號：Jerry 1.0
- 開發人員： Jerry


In [ ]:
#test = open("data/test.txt","w",encoding='UTF-8')

In [ ]:
# asb
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=99"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True)   #0 - 29

delay_choices = [1]#, 3, 5, 7,20]  #延遲的秒數
delay = random.choice(delay_choices)  #隨機選取秒數

for j in range(10):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=true&limit=99&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 101 == 0:
            print('第',j,'次執行，data reading...',i,'last',last,'chocice',delay)
    delay = random.choice(delay_choices)  #隨機選取秒數
    time.sleep(delay)  #延遲

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

In [ ]:
# 熱門美妝版測試 (100筆)
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']),ignore_index=True)   #0 - 29
    if i % 20 == 0:
        print('data reading...',i)
print(df.shape)
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色
# 235066991

In [ ]:
#last = str(int(df['ID'].tail(1))) #last dt
#last

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df['ID'].tail(1))) # 找出爬出資料的最後一筆ID
    url = 'https://www.dcard.tw/_api/posts?popular=true&before=' + last
    #url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 10 == 0:
            print('第',j,'次執行，data reading...',i)
            
print(df.shape)
df

In [ ]:
last = str(int(df['ID'].tail(1)))
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df

In [ ]:
# 熱門美妝版測試 (100筆)

for i in range(10):
    last = str(int(df['ID'].tail(1)))
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    df = pd.DataFrame()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']),ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
    print(df.shape)

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df

# 99	235031315	#請益 #問 LA girl 遮瑕挑色

In [48]:
last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID '235031315'
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)


df

AttributeError: 'DataFrame' object has no attribute 'ID'

In [ ]:

#df

In [ ]:
##ok ^
for j in range(3):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)
#df

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
print(df.shape)
df


In [ ]:
df.to_excel('data/Dcard_markeup.excel', index=False)

欄位	說明	備註
ID	編號	
title	標題	
content	內容	
excerpt	摘要	
createdAt	發布時間	
updatedAt	更新時間	
commentCount	留言數	
forumName	分類	中文
forumAlias	分類	英文
gender	性別	
likeCount	心情數量	
reactions	心情細項	把以上心情細分為「愛心」、「哈哈」、「跪」、「森77」、「驚訝」等類型
topics	標籤	

In [ ]:
# 固定碼測試
def Catch_test(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )
Catch_test(235020702)